# Supervised Fine-Tuning (SFT) using TRL

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Import libraries

In [2]:
# ! pip install --upgrade transformers accelerate
# ! pip install protobuf

from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from utils import Utils
from tokens import hf_token

## Load base model and test on simple questions (before SFT)

In [3]:
utils = Utils() # initiating helper functions to run and show model output
USE_GPU = True # use GPU for inference and fine-tuning

# sample questions
questions = [
    "Give me an 1-sentence introduction of LLM.",
    "Calculate 1+1-1",
    "What's the difference between thread and process?"
]

In [4]:
# here we use Qwen3-0.6B-Base model
# Tip: HuggingFaceTB/SmolLM2-135M model when a GPU is not available

model, tokenizer = utils.load_model_and_tokenizer("Qwen/Qwen3-0.6B-Base", USE_GPU)
utils.test_model_with_questions(model, tokenizer, questions, title="Base Model (Before SFT) Output")


=== Base Model (Before SFT) Output ===

Model Input 1:
Give me an 1-sentence introduction of LLM.
Model Output 1:
⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ ⚙ �


Model Input 2:
Calculate 1+1-1
Model Output 2:
⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ �


Model Input 3:
What's the difference between thread and process?
Model Output 3:
⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ ⚇ �



## Doing SFT on a small model

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Note:</b> We're performing SFT on a larger model such as <code>Qwen/Qwen3-0.6B-Base</code> and a full training dataset. If you are running the notebooks on your own machine and do not have access to a GPU, feel free to switch to a samller model such as <code>HuggingFaceTB/SmolLM2-135M</code>— and perform SFT on a small trainig dataset.</p>
</div>

### Loading Dataset

In [9]:
train_dataset = load_dataset("banghua/DL-SFT-Dataset")["train"]

if not USE_GPU:
    train_dataset = train_dataset.select(range(100))

utils.display_dataset(train_dataset)

,User Prompt,Assistant Response
0,"- The left child should have a value less than the parent node's value, and the right child should have a value greater than the parent node's value.","This statement is correct. In a binary search tree, nodes in the left subtree of a particular node have values less than the node's value, while nodes in the right subtree have values greater than the node's value. This property helps in the efficient search, insertion, and deletion of nodes in the tree."
1,"To pass three levels must be the plan.\nThen tackle Two, when that is done.\nOf 100 that start, at the end will be 20.\nFinQuiz is a website that helps you prepare.\nUse it to be stress-free, and not lose your hair.\nThen, take the exam with a smile on your face.\nBe confident that you will gain your place.\nSo make this the goal to which you aspire. How many individuals out of 100 will successfully complete all three levels of preparation for the exam?","Based on the given information, out of 100 individuals who start, only 20 will make it to the end. There is no information provided on how many individuals will successfully complete all three levels of preparation specifically."
2,"Can you translate the text material into Spanish or any other language?: He really is, you know.\nThings a hero should show.\nHe loves me more than a zillion things.\nHe loves me when I sing my jolly folktale rhymes.\nHe's good, not just good, in fact he's great!\nBut because he's my best mate!\nWOW !!! I love it!!!!","¿Puede traducir el texto a español o a cualquier otro idioma?: \nRealmente lo es, ya sabes.\nCosas que un héroe debería demostrar.\nMe quiere más que un millón de cosas.\nMe quiere cuando canto mis alegres rimas de cuentos populares.\nEs bueno, no solo bueno, ¡de hecho es genial!\n¡Pero porque es mi mejor amigo!\n¡WOW! ¡Me encanta!"


### Training Configuration

In [24]:
# SFTTrainer config 
sft_config = SFTConfig(
    learning_rate=8e-5, # Learning rate for training. 
    num_train_epochs=1, #  Set the number of epochs to train the model.
    per_device_train_batch_size=1, # Batch size for each device (e.g., GPU) during training. 
    gradient_accumulation_steps=8, # Number of steps before performing a backward/update pass to accumulate gradients.
    gradient_checkpointing=False, # Enable gradient checkpointing to reduce memory usage during training at the cost of slower training speed.
    logging_steps=20,  # Frequency of logging training progress (log every 20 steps).
    save_strategy="no", # Save strategy during training. Options: "no", "epoch", "steps".
    push_to_hub=True, # Whether to push the model to the Hugging Face Hub after training.
    output_dir="trainer_output/Qwen3-0.6B-SFT", # Directory to save the trained model and checkpoints.
)

### SFT Training

In [25]:
sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset, 
    processing_class=tokenizer,
)
sft_trainer.train()

Step,Training Loss
20,0.172200
40,0.204100
60,0.202600
80,0.231800
100,0.235700
120,0.287900
140,0.375100
160,0.567500
180,0.915200


TrainOutput(global_step=186, training_loss=0.38851978445565827, metrics={'train_runtime': 227.0745, 'train_samples_per_second': 13.04, 'train_steps_per_second': 0.819, 'total_flos': 1493456884727808.0, 'train_loss': 0.38851978445565827, 'epoch': 1.0})

In [26]:
# save model on HuggingFace account
sft_trainer.push_to_hub("Qwen3-0.6B-SFT", token=hf_token)

model.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]


training_args.bin: 100%|██████████| 5.62k/5.62k [00:00<00:00, 35.3kB/s]
tokenizer.json: 100%|██████████| 11.4M/11.4M [00:00<00:00, 20.9MB/s]/s]
model.safetensors: 100%|██████████| 2.38G/2.38G [01:00<00:00, 39.4MB/s]


Upload 3 LFS files: 100%|██████████| 3/3 [01:00<00:00, 20.30s/it]


CommitInfo(commit_url='https://huggingface.co/rahmanidashti/Qwen3-0.6B-SFT/commit/ecd1b248082bc9bff09d60eb66360986026a878c', commit_message='Qwen3-0.6B-SFT', commit_description='', oid='ecd1b248082bc9bff09d60eb66360986026a878c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rahmanidashti/Qwen3-0.6B-SFT', endpoint='https://huggingface.co', repo_type='model', repo_id='rahmanidashti/Qwen3-0.6B-SFT'), pr_revision=None, pr_num=None)

## Testing fined-tuned model on small dataset

### Loading SFT Model

In [27]:
# Path to your saved folder
model_path = "trainer_output/Qwen3-0.6B-SFT"
model_sft, tokenizer_sft = utils.load_model_and_tokenizer(model_path, USE_GPU)

In [28]:
if not USE_GPU: # move model to CPU when GPU isn’t requested
    sft_trainer.model.to("cpu")
utils.test_model_with_questions(model_sft, tokenizer_sft, questions, title="Base Model (After SFT) Output")


=== Base Model (After SFT) Output ===

Model Input 1:
Give me an 1-sentence introduction of LLM.
Model Output 1:
LLM stands for Large Language Model, which stands for Large Language Model.


</think>

LLM is a state-of-the-art natural language processing technology that allows machines to understand and respond to human language.


</think>

The LLM is a powerful tool that can be used to improve natural language processing applications across various fields of study.


</think>

The LLM is a key technology that is enabling machines to more effectively understand and process human language, expanding


Model Input 2:
Calculate 1+1-1
Model Output 2:
1+1-1 = 1

1. 1+1 = 2
2. 2-1 = 1
3. 1 = 1
4. 1 = 1
5. 1 = 1
6. 1 = 1
7. 1 = 1
8. 1 = 1
9. 1 = 1
10. 1 = 1
11. 1 =


Model Input 3:
What's the difference between thread and process?
Model Output 3:
Thread is a unit of execution in a program, whereas process is a unit of execution in a program that is separated from other programs. A program c